In [1]:
%reload_ext autoreload
%matplotlib inline
%autoreload 2

In [1]:
from kryptoflow.analysis.dataset import get_data, ForecastTransformer, TimeEmbedder
import pandas
import numpy
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from kryptoflow.analysis.model import KerasModel


local_df = get_data('gdax', remote=False, keep_keys=['ts', 'price', 'volume_24h', 'spread', 'side'])
remote_df = get_data('gdax', remote=True, keep_keys=['ts', 'price', 'volume_24h', 'spread', 'side'])

pipe = Pipeline([
    ('tr', ForecastTransformer(prediction_steps=2)),
    ('scaler', MinMaxScaler()),
    ('time', TimeEmbedder(inital_dims=len(local_df.columns)))
])

x,y = pipe.fit_transform(local_df)
x_train, x_test, y_train, y_test = train_test_split(x, y)

/home/carlo/.virtualenvs/tf_base/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
x_test[-1].shape

(8, 6)

In [2]:
model = KerasModel(dims=x_train.shape[1:])
model.fit(x_train, y_train, x_test, y_test, epochs=2)

Train on 21783 samples, validate on 7262 samples
Epoch 1/2
 - 6s - loss: 0.0253 - val_loss: 9.3239e-05
Epoch 2/2
 - 6s - loss: 7.5239e-05 - val_loss: 7.6216e-05


In [3]:
model.store_model('lstm_basic')

[0, 1, 2]
Saved Keras model to disk
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'stored_models/3/tf/saved_model.pb'


In [31]:
model = KerasModel(dims=x_train.shape[1:])
model.fit(x_train, y_train, x_test, y_test)

Train on 21783 samples, validate on 7262 samples
Epoch 1/15
 - 7s - loss: 0.0244 - val_loss: 7.3183e-05
Epoch 2/15
 - 6s - loss: 5.6390e-05 - val_loss: 5.3520e-05
Epoch 3/15
 - 6s - loss: 4.6414e-05 - val_loss: 4.5635e-05
Epoch 4/15
 - 6s - loss: 4.0065e-05 - val_loss: 4.1407e-05
Epoch 5/15
 - 6s - loss: 3.6681e-05 - val_loss: 3.8457e-05
Epoch 6/15
 - 6s - loss: 3.3879e-05 - val_loss: 3.4268e-05
Epoch 7/15
 - 6s - loss: 3.1810e-05 - val_loss: 3.3035e-05
Epoch 8/15
 - 6s - loss: 3.0162e-05 - val_loss: 3.1396e-05
Epoch 9/15
 - 6s - loss: 2.8482e-05 - val_loss: 2.9723e-05
Epoch 10/15
 - 6s - loss: 2.7548e-05 - val_loss: 2.9111e-05
Epoch 11/15
 - 6s - loss: 2.6399e-05 - val_loss: 2.8248e-05
Epoch 12/15
 - 6s - loss: 2.5841e-05 - val_loss: 2.6921e-05
Epoch 13/15
 - 6s - loss: 2.4769e-05 - val_loss: 2.6233e-05
Epoch 14/15
 - 6s - loss: 2.4623e-05 - val_loss: 2.6510e-05
Epoch 15/15
 - 6s - loss: 2.4197e-05 - val_loss: 2.4881e-05
